In [1]:
import numpy as np
import matplotlib as plt
import pandas as pd

In [2]:
#read csv files that contain the training and testing data
DS1_train=pd.read_csv("out/A2.1/DS1_train.csv",header=None)
DS1_train=np.array(DS1_train)
DS1_test=pd.read_csv("out/A2.1/DS1_test.csv",header=None)
DS1_test=np.array(DS1_test)

In [3]:
#learn parameters using maximum likelihood approach
N0=0
N1=0
m0=np.zeros([1,20])
m1=np.zeros([1,20])

for j in range(0,2800): 
    if(DS1_train[j,20]==1):
        N1=N1+1
        m1=m1+DS1_train[int(j),:20]
    else:
        N0=N0+1
        m0=m0+DS1_train[int(j),:20]
     
    
pi=float(N0)/(N0+N1)
m0=np.divide(m0,N0)
m1=np.divide(m1,N1)

s0=np.zeros([20,20])
s1=np.zeros([20,20])



for k in range(0,2800):
    if(DS1_train[k,20]==1):
        s1=np.add(s1,np.matmul((DS1_train[k,0:20]-m1).T,((DS1_train[k,0:20]-m1))))
        
    else:
        s0=np.add(s0,np.matmul((DS1_train[k,0:20]-m0).T,((DS1_train[k,0:20]-m0))))

s0=s0/float(N0)
s1=s1/float(N1)
s=pi*s0+(1-pi)*s1

In [4]:
#build up the model's parameter and define the sigmoid function
def sigmoid(a):
    return 1.0/(1.0+np.exp(-a))

w0_0=-0.5*m0.dot(np.linalg.inv(s)).dot(m0.T)+0.5*m1.dot(np.linalg.inv(s)).dot(m1.T)+np.log(pi/(1-pi))
w_0=np.linalg.inv(s).dot((m0-m1).T)

w0_1=-0.5*m1.dot(np.linalg.inv(s)).dot(m1.T)+0.5*m0.dot(np.linalg.inv(s)).dot(m0.T)+np.log((1-pi)/pi)
w_1=np.linalg.inv(s).dot((m1-m0).T)

print "w0_0: ", np.ravel(w0_0),"\n","w0_1: ", np.ravel(w0_1),"\n","w_0: ","\n",np.ravel(w_0),"\n","w_1:","\n",np.ravel(w_1)

w0_0:  [ 26.09831444] 
w0_1:  [-26.09831444] 
w_0:  
[ 13.83781553  -8.13577882  -5.2890129   -3.11571921  -9.16358703
  -4.34955068  16.03045348 -22.79275583 -27.57560007   8.69481137
 -12.67621349 -11.75712713  14.65557009  12.17681873  -5.55571079
  12.53713866  28.09403341  -6.37317519  -0.43307619  -4.62221076] 
w_1: 
[-13.83781553   8.13577882   5.2890129    3.11571921   9.16358703
   4.34955068 -16.03045348  22.79275583  27.57560007  -8.69481137
  12.67621349  11.75712713 -14.65557009 -12.17681873   5.55571079
 -12.53713866 -28.09403341   6.37317519   0.43307619   4.62221076]


In [5]:
#Compute the best fit accuracy, precision, recall and F-measure
tp=0
tn=0
fp=0
fn=0
for r in range(0,1200):
    p0=sigmoid(w0_0+w_0.T.dot(DS1_test[r,0:20]))
    p1=sigmoid(w0_1+w_1.T.dot(DS1_test[r,0:20]))
    if (p1>p0):
        if(DS1_test[r,20]):
            tp=tp+1
        else:
            fp=fp+1
    else:
        if(DS1_test[r,20]):
            fn=fn+1
        else:
            tn=tn+1

acc=(tp+tn)/1200.0            
pre=tp/float(tp+fp)
rec=tp/float(tp+fn)
f_mes=(2.0*pre*rec)/float(pre+rec)

print tp+fp+tn+fn
print "Accuracy: ", acc, "\n","Precision: ", pre, "\n","Recall: ", rec, "\n","F_measure: ", f_mes

1200
Accuracy:  0.960833333333 
Precision:  0.961279461279 
Recall:  0.959663865546 
F_measure:  0.96047098402
